### Statistical Analyses
#### List of tasks accomplished in this Jupyter Notebook:
- Calculate if there are any differences among the six species for each of the ten navigational variables (Kruskal test)
- Calculate if the larvae of each species prefer each of the different treatment odors
- Check if data gathered has a normal distribution (to inform the types of statistical tests that can be conducted)
- Calculate if there are mortality differences across the larval species (Fisher's exact test for binary data)
- Calculate pairwise perMANOVA for each species against each other species (navigation variables) as well as pairwise tests of multivariate dispersion (ANOVA) for all of the ten navigation variables during the acclimation period. 

In [1]:
R.version # print R version for reference
library(vegan)

               _                           
platform       x86_64-apple-darwin13.4.0   
arch           x86_64                      
os             darwin13.4.0                
system         x86_64, darwin13.4.0        
status                                     
major          3                           
minor          6.0                         
year           2019                        
month          04                          
day            26                          
svn rev        76424                       
language       R                           
version.string R version 3.6.0 (2019-04-26)
nickname       Planting of a Tree          

Loading required package: permute
Loading required package: lattice
This is vegan 2.5-6


In [2]:
setwd("/Users/ghost/Documents/Gitrepos/aedes-aegypti-2020/data/")
method_n <- "holm"

species <- c("Aedes aegypti", "Aedes albopictus", "Anopheles arabiensis", 
             "Anopheles gambiae", "Culex quinquefasciatus", "Culex tarsalis")

In [3]:
# FIGURE S2 and FIGURE 2

# Adjust all tests for the same data using the Holm-Bonferroni correction
p <- "placeholder"
t <- "placeholder"

data <- read.csv('./trajectories/cleaned_animal_analyses_acclimation.csv')
data <- subset(data, data$dead!='yes')
data$species <- as.factor(data$species)

# Individual variables by species
resp <- kruskal.test(data$time_move_p~data$species)
p <- c(p, resp$p.value)
t <- c(t, "time_move_p by species")

resp <- kruskal.test(data$total_dist_m~data$species)
p <- c(p, resp$p.value)
t <- c(t, "total_dist_m by species")

resp <- kruskal.test(data$avg_speed_BL~data$species)
p <- c(p, resp$p.value)
t <- c(t, "avg_speed_BL by species")

resp <- kruskal.test(data$max_speed_BL~data$species)
p <- c(p, resp$p.value)
t <- c(t, "max_speed_BL by species")

resp <- kruskal.test(data$mean_speed_first_BL~data$species)
p <- c(p, resp$p.value)
t <- c(t, "mean_speed_first_BL by species")

resp <- kruskal.test(data$diff_speed_first_last_BL~data$species)
p <- c(p, resp$p.value)
t <- c(t, "diff_speed_first_last_BL by species")

resp <- kruskal.test(data$sharp_turns_p~data$species)
p <- c(p, resp$p.value)
t <- c(t, "sharp_turns_p by species")

resp <- kruskal.test(data$spirals~data$species)
p <- c(p, resp$p.value)
t <- c(t, "spirals by species")

resp <- kruskal.test(data$continuous~data$species)
p <- c(p, resp$p.value)
t <- c(t, "continuous by species")

resp <- kruskal.test(data$max_still_sec~data$species)
p <- c(p, resp$p.value)
t <- c(t, "max_still_sec by species")

resp <- kruskal.test(data$length_mm~data$species)
p <- c(p, resp$p.value)
t <- c(t, "length_mm by species")

"TRUE/FALSE: Larval length data has a normal distribution"
"All tests must pass to use statistical tests that assume normality"
for (specie in species){
    spec <- subset(data, species==specie)
    print(shapiro.test(spec$length_mm)$p.value>0.05)}

"TRUE/FALSE: Larval length data has a normal distribution"
"All tests must pass to use statistical tests that assume normality"
for (specie in species){
    spec <- subset(data, species==specie)
    print(shapiro.test(spec$spirals)$p.value>0.05)}

# remove placeholder values and print out p values
p <- p[-1]; 
t <- t[-1];
cat("Number of tests ran:", length(p))
p <- p.adjust(p, method=method_n)
d <- data.frame(t, p, p<0.05, p<0.01, p<0.001, p<0.0001)
d

[1] "TRUE/FALSE: Larval length data has a normal distribution"

[1] "All tests must pass to use statistical tests that assume normality"

[1] FALSE
[1] TRUE
[1] FALSE
[1] FALSE
[1] TRUE
[1] TRUE


[1] "TRUE/FALSE: Larval length data has a normal distribution"

[1] "All tests must pass to use statistical tests that assume normality"

[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
Number of tests ran: 11

t,p,p...0.05,p...0.01,p...0.001,p...1e.04
time_move_p by species,1.906564e-66,TRUE,TRUE,TRUE,TRUE
total_dist_m by species,6.343186e-34,TRUE,TRUE,TRUE,TRUE
avg_speed_BL by species,1.045708e-48,TRUE,TRUE,TRUE,TRUE
max_speed_BL by species,1.347426e-57,TRUE,TRUE,TRUE,TRUE
mean_speed_first_BL by species,1.408676e-24,TRUE,TRUE,TRUE,TRUE
diff_speed_first_last_BL by species,8.845618e-37,TRUE,TRUE,TRUE,TRUE
sharp_turns_p by species,4.902439e-34,TRUE,TRUE,TRUE,TRUE
spirals by species,2.749885e-48,TRUE,TRUE,TRUE,TRUE
continuous by species,5.502450e-55,TRUE,TRUE,TRUE,TRUE
max_still_sec by species,9.820142e-50,TRUE,TRUE,TRUE,TRUE


In [4]:
# FIGURE 4A

data <- read.csv('./trajectories/cleaned_animal_analyses_experiment.csv')
data <- subset(data, data$dead!='yes')
treatments <- c("05_percent_food", "100ul_quinine", '100ul_milliQ_water')

data$treatment_odor <- as.factor(data$treatment_odor)
data$sex <- as.factor(data$sex)
data$species <- as.factor(data$species)

for (specie in species){
    spec <- subset(data, species==specie)
    print(specie)
    ps <- 'test'
    ts <- 'test'
    ds <- 'test'
    for (treat in treatments){
        ss <- subset(spec, treatment_odor==treat)
        resp <- t.test(ss$A_median_conc, ss$E_median_conc, paired=TRUE, 
                        alternative="two.sided")
        diff <- mean(ss$E_median_conc) - mean(ss$A_median_conc)
        ps <- c(ps, resp$p.value)
        ts <- c(ts, treat)
        ds <- c(ds, diff)}
    # remove placeholder values and print out p values
    ps <- ps[-1]; 
    ts <- ts[-1];
    ds <- ds[-1];
    d <- data.frame(ts, ds, ps)
    print(d)}

[1] "Aedes aegypti"
                  ts                 ds                   ps
1    05_percent_food    19.872251984127 0.000239226386199883
2      100ul_quinine           -8.60797 9.32954928568418e-06
3 100ul_milliQ_water 0.0421336805555566    0.983732683815243
[1] "Aedes albopictus"
                  ts                ds                   ps
1    05_percent_food   31.630240530303 1.47068238867747e-06
2      100ul_quinine -10.9511153846154 8.55609867672397e-07
3 100ul_milliQ_water  3.91859782608696    0.286956744900889
[1] "Anopheles arabiensis"
                  ts                 ds                  ps
1    05_percent_food   12.5379717261905 0.00483017723835698
2      100ul_quinine  -7.02041087962963 0.00182271020816546
3 100ul_milliQ_water -0.412895114942529   0.874252434426427
[1] "Anopheles gambiae"
                  ts                 ds                 ps
1    05_percent_food   6.56975126262626 0.0429712937330738
2      100ul_quinine  -7.19119369369369 0.0011600986894442
3 100

In [5]:
# FIGURE 4B

data <- read.csv('./trajectories/cleaned_animal_analyses_experiment.csv')
data <- subset(data, data$dead!='yes')
treatments <- c("05_percent_food", "100ul_quinine", "100ul_milliQ_water")

data$treatment_odor <- as.factor(data$treatment_odor)
data$sex <- as.factor(data$sex)
data$species <- as.factor(data$species)

"TRUE/FALSE: Data queried has a normal distribution"
"All tests must pass to use statistical tests that assume normality"

for (specie in species){
    spec <- subset(data, species==specie)
    for (treat in treatments){
        spec_t <- subset(spec, treatment_odor==treat)
        print(shapiro.test(spec_t$E_discovery_time)$p.value>0.05)}}

for (specie in species){
    spec <- subset(data, species==specie)
    water <- subset(spec, treatment_odor=='100ul_milliQ_water')
    resp <- kruskal.test(spec$discovery_time_diff~spec$treatment_odor)
    
    print("----")
    print(specie)
    print(resp$p.value)
}

[1] "TRUE/FALSE: Data queried has a normal distribution"

[1] "All tests must pass to use statistical tests that assume normality"

[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] FALSE
[1] TRUE
[1] FALSE
[1] "----"
[1] "Aedes aegypti"
[1] 0.59852
[1] "----"
[1] "Aedes albopictus"
[1] 0.7122485
[1] "----"
[1] "Anopheles arabiensis"
[1] 0.5197619
[1] "----"
[1] "Anopheles gambiae"
[1] 0.2151414
[1] "----"
[1] "Culex quinquefasciatus"
[1] 0.6395367
[1] "----"
[1] "Culex tarsalis"
[1] 0.9325915


In [7]:
# FIGURE S2

# Adjust all tests for the same data using the Holm-Bonferroni correction
p <- "placeholder"
t <- "placeholder"

mortality <- read.csv('./experiment_IDs/species_mortality_table.csv',
                      header=TRUE, row.names=1)
mortality

# Fisher's exact test for mortality data, between species
resp <- fisher.test(mortality, simulate.p.value=TRUE)
p <- c(p, resp$p.value)
t <- c(t, paste("Species differences", sep=" "))

# Fisher's exact test for mortality data, between treatments for each species
data <- read.csv('./experiment_IDs/experiment_mortality_table.csv', header=TRUE)
for (specie in species){
    data_subset <- subset(data, species==specie, select=c(alive, dead))
    resp <- fisher.test(data_subset, simulate.p.value=TRUE)
    p <- c(p, resp$p.value)
    t <- c(t, paste(specie, sep=" "))}

# remove placeholder values and print out p values
p <- p[-1]; 
t <- t[-1];

# cat("Number of tests ran:", length(p))
p <- p.adjust(p, method=method_n)
d <- data.frame(t, p, p<0.05, p<0.01, p<0.001, p<0.0001)
print(d)

,alive,dead
Aedes aegypti,67,2
Aedes albopictus,70,7
Anopheles arabiensis,93,50
Anopheles coluzzii,108,5
Culex quinquefasciatus,110,21
Culex tarsalis,53,31


                       t           p p...0.05 p...0.01 p...0.001 p...1e.04
1    Species differences 0.003498251     TRUE     TRUE     FALSE     FALSE
2          Aedes aegypti 1.000000000    FALSE    FALSE     FALSE     FALSE
3       Aedes albopictus 1.000000000    FALSE    FALSE     FALSE     FALSE
4   Anopheles arabiensis 1.000000000    FALSE    FALSE     FALSE     FALSE
5      Anopheles gambiae 1.000000000    FALSE    FALSE     FALSE     FALSE
6 Culex quinquefasciatus 1.000000000    FALSE    FALSE     FALSE     FALSE
7         Culex tarsalis 1.000000000    FALSE    FALSE     FALSE     FALSE


In [9]:
# FIGURE 3
n <- 10000

# perMANOVA for all six species at once
data <- read.csv('./trajectories/cleaned_animal_analyses_acclimation.csv')
sp.data <- subset(data, select=-c(species, treatment_odor, sex, dead, length_mm))
sp.species <- subset(data, select=c(animal_ID, species))
rownames(sp.data) <- sp.data$animal_ID
sp.data <- subset(sp.data, select=-c(animal_ID))
rownames(sp.species) <- sp.species$animal_ID
sp.species <- subset(sp.species, select=-c(animal_ID))
sp.data.tra<-decostand(sp.data, method='standardize', margin='column', plot=F)
perm <- adonis(sp.data.tra~species, data=sp.species, 
               permutations=n, method='euclidean')
# perm[[1]]
print("----")
print("Comparison across all species")
perm
print("----")

# Adjust all tests for the same data using the Holm-Bonferroni correction
p <- "placeholder"
t <- "placeholder"
f <- "placeholder"

sp_pairs <- list()
sp_pairs[[1]] <- c('Aedes aegypti', 'Aedes albopictus')
sp_pairs[[2]] <- c('Aedes aegypti', 'Anopheles arabiensis')
sp_pairs[[3]] <- c('Aedes aegypti', 'Anopheles gambiae')
sp_pairs[[4]] <- c('Aedes aegypti', 'Culex tarsalis')
sp_pairs[[5]] <- c('Aedes aegypti', 'Culex quinquefasciatus')
sp_pairs[[6]] <- c('Aedes albopictus', 'Anopheles arabiensis')
sp_pairs[[7]] <- c('Aedes albopictus', 'Anopheles gambiae')
sp_pairs[[8]] <- c('Aedes albopictus', 'Culex tarsalis')
sp_pairs[[9]] <- c('Aedes albopictus', 'Culex quinquefasciatus')
sp_pairs[[10]] <- c('Anopheles arabiensis', 'Anopheles gambiae')
sp_pairs[[11]] <- c('Anopheles arabiensis', 'Culex tarsalis')
sp_pairs[[12]] <- c('Anopheles arabiensis', 'Culex quinquefasciatus')
sp_pairs[[13]] <- c('Anopheles gambiae', 'Culex tarsalis')
sp_pairs[[14]] <- c('Anopheles gambiae', 'Culex quinquefasciatus')
sp_pairs[[15]] <- c('Culex tarsalis', 'Culex quinquefasciatus')

for (pair in sp_pairs){
    data <- read.csv('./trajectories/cleaned_animal_analyses_acclimation.csv')
    data <- subset(data, species == pair[1] | species == pair[2])
    data$species <- as.factor(data$species)
    sp.data <- subset(data, select=-c(species, treatment_odor, sex, dead, length_mm))
    sp.species <- subset(data, select=c(animal_ID, species))
    rownames(sp.data) <- sp.data$animal_ID
    sp.data <- subset(sp.data, select=-c(animal_ID))
    rownames(sp.species) <- sp.species$animal_ID
    sp.species <- subset(sp.species, select=-c(animal_ID))
    sp.data.tra<-decostand(sp.data, method='standardize', margin='column', plot=F)
    perm <- adonis(sp.data.tra~species, data=sp.species, 
                   permutations=n, method='euclidean')
    p <- c(p, perm[[1]][1,6])
    f <- c(f, perm[[1]][1,4])
    t <- c(t, paste(c('PM', pair), collapse=', '))
}

for (pair in sp_pairs){
    data <- read.csv('./trajectories/cleaned_animal_analyses_acclimation.csv')
    data <- subset(data, species == pair[1] | species == pair[2])
    data$species <- as.factor(data$species)
    sp.data <- subset(data, select=-c(species, treatment_odor, sex, dead, length_mm))
    sp.species <- subset(data, select=c(animal_ID, species))
    rownames(sp.data) <- sp.data$animal_ID
    sp.data <- subset(sp.data, select=-c(animal_ID))
    rownames(sp.species) <- sp.species$animal_ID
    sp.species <- subset(sp.species, select=-c(animal_ID))
    sp.data.tra<-decostand(sp.data, method='standardize', margin='column', plot=F)
    sp.dist <- vegdist(sp.data.tra,"euclidean")
    sp.bdp <- betadisper(sp.dist, sp.species$species)
    an <- anova(sp.bdp)
    p <- c(p, an[1, 5])
    f <- c(f, an[1, 4])
    t <- c(t, paste(c('BDP', pair), collapse=', '))
}

# remove placeholder values and print out p values
p <- p[-1]; 
t <- t[-1];
f <- f[-1];

# cat("Number of tests ran:", length(p))
p <- p.adjust(p, method=method_n)
d <- data.frame(t, p, f)
print(d)

[1] "----"
[1] "Comparison across all species"



Call:
adonis(formula = sp.data.tra ~ species, data = sp.species, permutations = n,      method = "euclidean") 

Permutation: free
Number of permutations: 10000

Terms added sequentially (first to last)

           Df SumsOfSqs MeanSqs F.Model      R2    Pr(>F)    
species     5    2089.5  417.90  70.677 0.41458 9.999e-05 ***
Residuals 499    2950.5    5.91         0.58542              
Total     504    5040.0                 1.00000              
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "----"
                                                   t            p
1                PM, Aedes aegypti, Aedes albopictus 0.0026997300
2            PM, Aedes aegypti, Anopheles arabiensis 0.0026997300
3               PM, Aedes aegypti, Anopheles gambiae 0.0026997300
4                  PM, Aedes aegypti, Culex tarsalis 0.0026997300
5          PM, Aedes aegypti, Culex quinquefasciatus 0.0026997300
6         PM, Aedes albopictus, Anopheles arabiensis 0.0026997300
7            PM, Aedes albopictus, Anopheles gambiae 0.0026997300
8               PM, Aedes albopictus, Culex tarsalis 0.0026997300
9       PM, Aedes albopictus, Culex quinquefasciatus 0.0026997300
10       PM, Anopheles arabiensis, Anopheles gambiae 0.0026997300
11          PM, Anopheles arabiensis, Culex tarsalis 0.0026997300
12  PM, Anopheles arabiensis, Culex quinquefasciatus 0.0026997300
13             PM, Anopheles gambiae, Culex tarsalis 0.0026997300
14     PM, Anopheles gambiae, Culex quinquefasciatus 0.0026997300

In [ ]:
# Test pairwise code for just one pair
pair <- c('Anopheles gambiae', 'Culex quinquefasciatus')

data <- read.csv('./trajectories/cleaned_animal_analyses_acclimation.csv')
data <- subset(data, species == pair[1] | species == pair[2])
data$species <- as.factor(data$species)
sp.data <- subset(data, select=-c(species, treatment_odor, sex, dead, length_mm))
sp.species <- subset(data, select=c(animal_ID, species))
rownames(sp.data) <- sp.data$animal_ID
sp.data <- subset(sp.data, select=-c(animal_ID))
rownames(sp.species) <- sp.species$animal_ID
sp.species <- subset(sp.species, select=-c(animal_ID))
sp.data.tra<-decostand(sp.data, method='standardize', margin='column', plot=F)
perm <- adonis(sp.data.tra~species, data=sp.species, 
               permutations=100, method='euclidean')
PM_p <- perm[[1]][1,6]
PM_f <- perm[[1]][1,4]

sp.dist <- vegdist(sp.data.tra,"euclidean")
sp.bdp <- betadisper(sp.dist, sp.species$species)
an <- anova(sp.bdp)

BDP_p <- an[1, 5]
BDP_f <- an[1, 4]

# Print everything 
perm[[1]]
PM_p
PM_f
an
BDP_p
BDP_f